In [3]:
# Importing libraries
import pandas as pd
from scipy.stats import f_oneway

In [158]:
## Reading and cleaning data

# Reading data
cpm_ngs_pivot = pd.read_csv("Data/cpm_NGS_1_deiden.csv", index_col=0)
cpm_ngs_pivot = cpm_ngs_pivot.rename(columns={'ENSEMBL':'ensembl_gene_id','ENTREZID':'entrez_gene_id',"GENENAME":'gene','SYMBOL':'symbol'})

# Seperate Gene ID Lookup table and expression data table
geneID = cpm_ngs_pivot[['ensembl_gene_id', 'entrez_gene_id','gene', 'symbol']].drop_duplicates()

cpm_ngs_pivot = cpm_ngs_pivot.drop(columns=['entrez_gene_id','gene', 'symbol'])

cpm_ngs = cpm_ngs_pivot.melt(id_vars=['ensembl_gene_id'],
                             var_name='drug_sub', value_name='expression_val')

# Creating seperate drug and subject columns
cpm_ngs['drug'] = cpm_ngs['drug_sub'].str[:-6]
cpm_ngs['subject'] = cpm_ngs['drug_sub'].str[-1:]

# String formatting
cpm_ngs['ensembl_gene_id'].str.strip()
cpm_ngs['subject'].str.strip()
cpm_ngs['drug'] = cpm_ngs['drug'].replace({'Drug A ': 'A', 'Drug B ': 'B', 'Drug C': 'C', 'Saline': 'S'})

# Ordering columns
cpm_ngs = cpm_ngs[['ensembl_gene_id','drug','subject','expression_val']]

# Printing first few rows
cpm_ngs.head()


,ensembl_gene_id,drug,subject,expression_val
0,ENSMUSG00000000001,A,1,30.418821
1,ENSMUSG00000000003,A,1,0.000000
2,ENSMUSG00000000028,A,1,0.790703
3,ENSMUSG00000000031,A,1,0.139536
4,ENSMUSG00000000037,A,1,0.930239


In [168]:
# ANOVA test

genes = []
f_vals = []
p_vals = []

for gene in cpm_ngs['ensembl_gene_id'].drop_duplicates():
    geneFilter = cpm_ngs[cpm_ngs['ensembl_gene_id'] == gene]

    drugA = geneFilter.loc[geneFilter['drug'] == 'A', 'expression_val'].tolist()
    drugB = geneFilter.loc[geneFilter['drug'] == 'B', 'expression_val'].tolist()
    drugC = geneFilter.loc[geneFilter['drug'] == 'C', 'expression_val'].tolist()
    saline = geneFilter.loc[geneFilter['drug'] == 'S', 'expression_val'].tolist()
    f_val, p_val = f_oneway(drugA, drugB, drugC, saline)

    genes.append(gene)
    f_vals.append(f_val)
    p_vals.append(p_val)

    cpm_ngs = cpm_ngs[cpm_ngs['ensembl_gene_id'] != gene]


# Creating new dataframe of signifiance of mean differences for each gene
mean_diff_sig = pd.DataFrame({'gene': genes, 'f_val':f_vals, 'p_val':p_vals})

# Significance column indicating 1 if p_val < 0.05
mean_diff_sig['significance'] = (mean_diff_sig['p_val'] < 0.05).astype(int)

# Export table
mean_diff_sig.to_csv("Output/mean_diff_sig.csv", index=False)

# Show the table
mean_diff_sig


/home/sunzid/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4167: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))


,gene,f_val,p_val,significance
0,ENSMUSG00000000001,1.246670,0.325792,0
1,ENSMUSG00000000003,NaN,NaN,0
2,ENSMUSG00000000028,1.120112,0.370303,0
3,ENSMUSG00000000031,0.743971,0.541406,0
4,ENSMUSG00000000037,0.437300,0.729362,0
...,...,...,...,...
55531,ENSMUSG00000118389,NaN,NaN,0
55532,ENSMUSG00000118390,NaN,NaN,0
55533,ENSMUSG00000118391,0.574891,0.639748,0
55534,ENSMUSG00000118392,NaN,NaN,0


In [204]:
# Creating new dataframe of signifiance of mean differences for each gene
mean_diff_sig = pd.DataFrame({'gene': genes, 'f_val':f_vals, 'p_val':p_vals})

# Significance column indicating 1 if p_val < 0.05
mean_diff_sig['p_sig'] = (cpm_ngs['p_val'] < 0.05).astype(int)
mean_diff_sig['f_sig'] = (cpm_ngs['f_val'] > 4.0).astype(int)
mean_diff_sig['sig'] = (mean_diff_sig['p_sig'] & mean_diff_sig['f_sig']).astype(int)

sig_mean_diff = mean_diff_sig[mean_diff_sig['sig'] == 1]
sig_mean_diff = sig_mean_diff[['gene', 'f_val',	'p_val', 'sig']]

# Show the table
print(sig_mean_diff.shape)
sig_mean_diff

KeyError: 'p_val'